In [4]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import re
import pickle

import tensorflow as tf


import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [5]:
cornell_data = pickle.load(open('cornell_data.pkl','rb'))

In [6]:
cornell_data[1]['request'] + " " + cornell_data[1]['reply']

'not the hacking and gagging and spitting part please okay then how bout we try out some french cuisine saturday night'

In [8]:
#Create vocabulary 
vocab = {}
questions = []
answers = []
for pair in cornell_data:
    text = pair['request'] + " " + pair['reply']
    questions.append(pair['request'])
    answers.append(pair['reply'])
    for word in text.split():
        lword = lemmatizer.lemmatize(word)
        if lword not in vocab:
            vocab[lword] = 1
        else:
            vocab[lword] += 1

In [9]:
len(vocab)

40264

In [10]:
vocab

{'well': 8402,
 'i': 108594,
 'thought': 2602,
 'we': 21234,
 'would': 9871,
 'start': 868,
 'with': 11184,
 'pronunciation': 1,
 'if': 7954,
 'that': 33966,
 'is': 74144,
 'okay': 4277,
 'you': 121591,
 'not': 17883,
 'the': 63464,
 'hacking': 9,
 'and': 22503,
 'gagging': 7,
 'spitting': 12,
 'part': 655,
 'please': 2336,
 'then': 4283,
 'how': 10313,
 'bout': 271,
 'try': 1233,
 'out': 8753,
 'some': 4369,
 'french': 156,
 'cuisine': 7,
 'saturday': 100,
 'night': 2139,
 'are': 31370,
 'asking': 407,
 'me': 24653,
 'so': 9488,
 'cute': 170,
 'what': 36066,
 'your': 15728,
 'name': 2249,
 'again': 1823,
 'forget': 802,
 'it': 50816,
 'no': 18376,
 'my': 14806,
 'fault': 245,
 'didnt': 5110,
 'have': 23486,
 'a': 51909,
 'proper': 79,
 'introduction': 12,
 'cameron': 28,
 'gosh': 76,
 'only': 2512,
 'could': 4137,
 'find': 2046,
 'kat': 34,
 'boyfriend': 174,
 'let': 4759,
 'see': 6255,
 'can': 8292,
 'do': 20045,
 'cesc': 1,
 'ma': 209,
 'tete': 1,
 'this': 17168,
 'head': 795,
 'rig

In [12]:
#Remove rare words
threshold = 10
c = 0

for word, count in vocab.items():
    if count >= threshold:
        c += 1

In [13]:
print("Total vocab: %d" % len(vocab))
print("Usable vocab: %d" % c)

Total vocab: 40264
Usable vocab: 8156


In [122]:
#Assign unique int value to each word in vocab
questions_vocab_to_int = {}
answers_vocab_to_int = {}
vocab_to_int = {}

word_num = 0

for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        answers_vocab_to_int[word] = word_num
        vocab_to_int[word] = word_num
        word_num += 1

In [123]:
#Add unique tokens to vocab
codes = ['<PAD>', '<EOS>', '<UNK>', '<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int) + 1
    answers_vocab_to_int[code] = len(answers_vocab_to_int) + 1
    vocab_to_int[code] = len(vocab_to_int) + 1

In [124]:
#Converting from word->int to int->word
questions_int_to_vocab = {v_i: word for word,v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: word for word,v_i in answers_vocab_to_int.items()}
int_to_vocab = {v_i: word for word, v_i in vocab_to_int.items()}

In [125]:
#Add <EOS> token to end of answers
for i in range(len(cornell_data)):
    cornell_data[i]['reply'] += ' <EOS>'

In [126]:
#Replace words that are not in vocab with <UNK>

vocab_int = []

for pair in cornell_data:
    text = pair['request'] + ' ' + pair['reply']
    ints = []
    for word in text.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    vocab_int.append(ints)

In [137]:
questions_int = []
answers_int = []
for pair in cornell_data:
    ints = []
    for word in pair['request'].split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    questions_int.append(ints)
    ints = []
    for word in pair['reply'].split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    answers_int.append(ints)

In [139]:
print(len(cornell_data), len(vocab_int), len(questions_int), len(answers_int))

167497 167497 167497 167497


In [142]:
sorted_questions = []
sorted_answers = []
sorted_data = []

max_len = 20

for length in range(1, max_len + 1):
    for i in enumerate(questions_int):
        if(len(i[1]) == length):
            sorted_data += [{'request': questions_int[i[0]], 'reply': answers_int[i[0]]}]

In [158]:
sorted_data[75000]['request']

[3, 42, 460, 601, 58]

497

In [179]:
query = 'are you serious'
hay = []
for x in query.split():
    hay.append(vocab_to_int[x])
for data in sorted_data:
    if data['request'] == hay:
        question=""
        answer =""
        for idx in data['request']:
            question += " " + int_to_vocab[idx]
        for idx in data['reply']:
            answer += " " + int_to_vocab[idx]
        print(question + " - " + answer)

 are you serious -  yeah he said that <EOS>
 are you serious -  huh imagine that looks like i am <EOS>
 are you serious -  i dont even know whether a defense is possible <EOS>
 are you serious -  yeah i am out <EOS>
 are you serious -  not ever <EOS>
 are you serious -  how long have you been smoking <EOS>
 are you serious -  that is history fletch you owe us nine hundred and eighteen dollars <EOS>
 are you serious -  of course look i have got some missing persons <UNK> evidence of a large <UNK> <EOS>
 are you serious -  forget it it doesnt matter <EOS>
 are you serious -  yeah i think so <EOS>
 are you serious -  have you ever known me not to be <EOS>
 are you serious -  yeah no more i am finished with that <UNK> <EOS>
 are you serious -  give him the <UNK> <EOS>


## -----------------------------------------------------------------------------------------------------
## -----------------------------------------------------------------------------------------------------
## -----------------------------------------------------------------------------------------------------